# Loan predictions

## Problem Statement

We want to automate the loan eligibility process based on customer details that are provided as online application forms are being filled. You can find the dataset [here](https://drive.google.com/file/d/1h_jl9xqqqHflI5PsuiQd_soNYxzFfjKw/view?usp=sharing). These details concern the customer's Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and other things as well. 

|Variable| Description|
|: ------------- |:-------------|
|Loan_ID| Unique Loan ID|
|Gender| Male/ Female|
|Married| Applicant married (Y/N)|
|Dependents| Number of dependents|
|Education| Applicant Education (Graduate/ Under Graduate)|
|Self_Employed| Self employed (Y/N)|
|ApplicantIncome| Applicant income|
|CoapplicantIncome| Coapplicant income|
|LoanAmount| Loan amount in thousands|
|Loan_Amount_Term| Term of loan in months|
|Credit_History| credit history meets guidelines|
|Property_Area| Urban/ Semi Urban/ Rural|
|Loan_Status| Loan approved (Y/N)



### Explore the problem in following stages:

1. Hypothesis Generation – understanding the problem better by brainstorming possible factors that can impact the outcome
2. Data Exploration – looking at categorical and continuous feature summaries and making inferences about the data.
3. Data Cleaning – imputing missing values in the data and checking for outliers
4. Feature Engineering – modifying existing variables and creating new ones for analysis
5. Model Building – making predictive models on the data

## 1. Hypothesis Generation

Generating a hypothesis is a major step in the process of analyzing data. This involves understanding the problem and formulating a meaningful hypothesis about what could potentially have a good impact on the outcome. This is done BEFORE looking at the data, and we end up creating a laundry list of the different analyses which we can potentially perform if data is available.

#### Possible hypotheses
Which applicants are more likely to get a loan

1. Applicants having a credit history 
2. Applicants with higher applicant and co-applicant incomes
3. Applicants with higher education level
4. Properties in urban areas with high growth perspectives
+5. Applicants with higher income per person in the family (considering #of dependents)
+6. Applicants with lower loan amount in application
+7. Applicants with high savings (data not provided here)

Do more brainstorming and create some hypotheses of your own. Remember that the data might not be sufficient to test all of these, but forming these enables a better understanding of the problem.

## 2. Data Exploration
Let's do some basic data exploration here and come up with some inferences about the data. Go ahead and try to figure out some irregularities and address them in the next section. 

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

df = pd.read_csv("data.csv") 
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

One of the key challenges in any data set are missing values. Lets start by checking which columns contain missing values.

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

Look at some basic statistics for numerical variables.

In [ ]:
df.describe()

In [ ]:
df[df['LoanAmount'].isnull()==True].sample()

1. How many applicants have a `Credit_History`? (`Credit_History` has value 1 for those who have a credit history and 0 otherwise)
2. Is the `ApplicantIncome` distribution in line with your expectation? Similarly, what about `CoapplicantIncome`?
3. Tip: Can you see a possible skewness in the data by comparing the mean to the median, i.e. the 50% figure of a feature.



Let's discuss nominal (categorical) variable. Look at the number of unique values in each of them.

In [ ]:
df['TotalIncome'] = df['ApplicantIncome'] + df['CoapplicantIncome']

In [ ]:
# df['Income/family'] = df['TotalIncome']/(df['Dependents']+1) +1 (if married is yes) 

In [ ]:
df['Credit_History'].value_counts()

In [ ]:
df['Credit_History'].isnull().sum() # to fill in '0' or delete all

In [ ]:
df['Gender'].value_counts()

In [ ]:
df['Married'].value_counts()

In [ ]:
df['Dependents'].value_counts()

In [ ]:
df['Education'].value_counts()

In [ ]:
df['Self_Employed'].value_counts()

In [ ]:
df['Property_Area'].value_counts()

In [ ]:
import seaborn as sns
sns.displot(x=df['ApplicantIncome'], col=df['Loan_Status'])

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize =(10, 7))

# Creating axes instance
ax = fig.add_axes([0, 0, 1, 1])

data = df[['ApplicantIncome','CoapplicantIncome']]
# Creating plot
ax.boxplot(data)
# show plot
plt.show()

Explore further using the frequency of different categories in each nominal variable. Exclude the ID obvious reasons.

In [ ]:
df[df["Loan_Status"] == 'Y'].describe()

In [ ]:
df[df["Loan_Status"] == 'N'].describe()

### Distribution analysis

Study distribution of various variables. Plot the histogram of ApplicantIncome, try different number of bins.



In [ ]:
# Plot
fig, ax =plt.subplots(1,2)
sns.displot(df['ApplicantIncome'], bins=50, ax=ax[0])
sns.displot(df['CoapplicantIncome'], bins=50, ax=ax[1])


Look at box plots to understand the distributions. 

Look at the distribution of income segregated  by `Education`

In [ ]:
import seaborn as sns
sns.displot(x=df['ApplicantIncome'], col=df['Credit_History'])

Look at the histogram and boxplot of LoanAmount

In [ ]:
sns.displot(x=df['LoanAmount'])

In [ ]:
import seaborn as sns
sns.displot(x=df['TotalIncome'], col=df['Gender'])

In [ ]:
sns.displot(x=df['TotalIncome'], y=df['LoanAmount'])

In [ ]:
ax = sns.boxplot(x='Loan_Status', y ='TotalIncome', data=df)

There might be some extreme values. Both `ApplicantIncome` and `LoanAmount` require some amount of data munging. `LoanAmount` has missing and well as extreme values values, while `ApplicantIncome` has a few extreme values, which demand deeper understanding. 

### Categorical variable analysis

Try to understand categorical variables in more details using `pandas.DataFrame.pivot_table` and some visualizations.

In [ ]:
import numpy as np
table = pd.pivot_table(df, values='ApplicantIncome', index=['Gender','Credit_History','Property_Area'], aggfunc=np.mean)
table

In [ ]:
table2 = pd.pivot_table(df, values='LoanAmount', index=['Married','Self_Employed'], aggfunc=np.mean)
table2

In [ ]:
df['Self_Employed'][df['Self_Employed'].notnull()].value_counts()

In [ ]:
df['Loan_Status'][(df['Self_Employed']=='No')].value_counts()

In [ ]:
#df Self_Employed = 'Yes' and Loan_Status = 'Y'
print("Self_Employed = 'Yes' and Loan_Status = 'Y':", 56/82)
print("Self_Employed = 'No' and Loan_Status = 'Y':", 343/500)

In [ ]:
# Self_employed seems not so relevant with loan approval?

## 3. Data Cleaning

This step typically involves imputing missing values and treating outliers. 

### Imputing Missing Values

Missing values may not always be NaNs. For instance, the `Loan_Amount_Term` might be 0, which does not make sense.



Impute missing values for all columns. Use the values which you find most meaningful (mean, mode, median, zero.... maybe different mean values for different groups)

In [ ]:
table3 = pd.pivot_table(df, values='Loan_Status', index=['Credit_History'], aggfunc='count')
table3

In [ ]:
df = df.fillna({'Credit_History':0, 'Gender': 'Male','Dependents': '0',
                'Loan_Amount_Term':df['Loan_Amount_Term'].mean(), 'LoanAmount':df['LoanAmount'].mean()})

In [ ]:
df.info()

In [ ]:
df = df.drop(columns=['Self_Employed'])
df = df.dropna()

In [ ]:
df.loc[df.Dependents == '3+', 'Dependents'] = '3'
df

### Extreme values
Try a log transformation to get rid of the extreme values in `LoanAmount`. Plot the histogram before and after the transformation

In [ ]:
# histogram before transformation
sns.displot(df['LoanAmount'])

In [ ]:
loan_log = np.log(df['LoanAmount'])

In [ ]:
# histogram after transformation
sns.displot(loan_log)

Combine both incomes as total income and take a log transformation of the same.

In [ ]:
#histogram before transformation
sns.displot(df['TotalIncome'])

In [ ]:
#histogram after transformation
income_log = np.log(df['TotalIncome'])
sns.displot(income_log)

### Before using pipeline - manual cleansing

In [ ]:
y = df['Loan_Status'].values
X = df.drop(columns=['Loan_Status','Loan_ID'])

In [ ]:
X_cat = X[['Gender','Married','Dependents','Education','Credit_History','Property_Area']]
X_num = X[['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','TotalIncome']]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
X_cat_trans = enc.fit_transform(X_cat)
X_cat_trans

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_num_scaled = scaler.fit_transform(X_num)
X_num_scaled

In [ ]:
X_con = np.concatenate((X_num_scaled, X_cat_trans), axis=1)

## 4. Building a Predictive Model

In [ ]:
# 1. scaling
# 2. building a model (Logistic Regression, Naive Bayes, Decision Tree/Random Forest, SVM, KNN?)

In [ ]:
# Data Splitting 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_con, y, random_state=27)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support, f1_score

logreg_clf = LogisticRegression()
LDA_clf = LinearDiscriminantAnalysis()
KNN_clf = KNeighborsClassifier()
NB_clf = GaussianNB()
DT_clf = DecisionTreeClassifier()
SVM_clf = SVC()


In [ ]:
logreg_clf.fit(X_con,y)

In [ ]:
y_pred = logreg_clf.predict(X_test)

In [ ]:

score = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print(f'Test set precision, recall, f-score: {score}')

In [ ]:
LDA_clf.fit(X_con,y)
y_pred_LDA = LDA_clf.predict(X_test)
score = precision_recall_fscore_support(y_test, y_pred_LDA, average='weighted')
print(f'Test set precision, recall, f-score: {score}')

In [ ]:
KNN_clf.fit(X_con,y)
y_pred_KNN = KNN_clf.predict(X_test)
score = precision_recall_fscore_support(y_test, y_pred_KNN, average='weighted')
print(f'Test set precision, recall, f-score: {score}')

In [ ]:
NB_clf.fit(X_con,y)
y_pred_NB = NB_clf.predict(X_test)
score = precision_recall_fscore_support(y_test, y_pred_NB, average='weighted')
print(f'Test set precision, recall, f-score: {score}')

In [ ]:
DT_clf.fit(X_con,y)
y_pred_DT = DT_clf.predict(X_test)
score = precision_recall_fscore_support(y_test, y_pred_DT, average='weighted')
print(f'Test set precision, recall, f-score: {score}')

In [ ]:
SVM_clf.fit(X_con,y)
y_pred_SVM = SVM_clf.predict(X_test)
score = precision_recall_fscore_support(y_test, y_pred_SVM, average='weighted')
print(f'Test set precision, recall, f-score: {score}')

Try paramater grid search to improve the results

In [ ]:
# will do it after pipeline!!!! 

## 5. Using Pipeline
If you didn't use pipelines before, transform your data prep, feat. engineering and modeling steps into Pipeline. It will be helpful for deployment.

The goal here is to create the pipeline that will take one row of our dataset and predict the probability of being granted a loan.

`pipeline.predict(x)`

In [ ]:
df_Loan = pd.read_csv("data.csv") 

### Pipeline order 
##### Preprocessing 
1. Creating a new column 'Total income' 
2. Missing value fill in
3. Log transformation

##### Main pipeline
 - Preprocessing
 - Numerical & Categorical value split and One Hot Encoding for Categorical values
 - classifier 

##### After pipeline
 - grid search 
 - pickling

In [ ]:
# Preprocessing_1. new column
df_Loan['TotalIncome'] = df_Loan['ApplicantIncome'] + df_Loan['CoapplicantIncome']
df_Loan = df_Loan.drop('Loan_ID', axis=1)
df_Loan.info()

In [ ]:
X = df_Loan.drop(['Loan_Status','ApplicantIncome','CoapplicantIncome'], axis=1)
y = df_Loan['Loan_Status'].values

In [ ]:
# Data Splitting 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=27)

###### Originally I took the steps above by using column transformer each steps, but it turns out that once you use column transformer it returns np.array, so you can't specify column names afterwards. Thus, I changed the way to use column transformer only once. 

In [ ]:
import pandas as pd
#Own data-frame transformer class used in pipeline
class DataframeTransformer:
    def __init__(self, func):
        self.func = func
    
    def transform(self, input_df, **transform_param):
        return self.func(input_df)
       
    def fit(self, X, y=None, **fit_params):
        return self

def create_total_income_feature(input_df):
    input_df['total_income'] = input_df['ApplicantIncome'] + input_df['CoapplicantIncome']
    return input_df
    
def to_dataframe(array):
    columns=['LoanAmount','TotalIncome', 'Gender','Dependents','Self_Employed','Credit_History',
             'Married','Education', 'Loan_Amount_Term','Property_Area' ]
    df = pd.DataFrame(array, columns=columns)
    convert_dict = {'LoanAmount':'float64','TotalIncome':'float64'}
    df = df.astype(convert_dict)
    return df


In [ ]:
# Pipeline 
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import precision_recall_fscore_support, f1_score

fillna_trans = ColumnTransformer([
    ('fill_avg', SimpleImputer(strategy='mean'), ['LoanAmount', 'TotalIncome']),
    ('fill_mode', SimpleImputer(strategy='most_frequent'), ['Gender','Dependents','Self_Employed']),
    ('fill_zero', SimpleImputer(strategy='constant', fill_value = 0), ['Credit_History'])], remainder='passthrough')

log_trans = ColumnTransformer([('log_trans', FunctionTransformer(np.log), ['LoanAmount', 'TotalIncome'])], 
                              remainder='passthrough')
                             
preprocessing = Pipeline([
    ('fillna_trans', fillna_trans),
    ('to_dataframe', DataframeTransformer(to_dataframe)),
    ('log_trans', log_trans),
    ('to_dataframe2', DataframeTransformer(to_dataframe))    
])

preprocessing

In [ ]:


pipeline = Pipeline(steps=[('preprocessing', preprocessing),
                           ('encoding', OneHotEncoder(sparse=False,handle_unknown='ignore')),
                           ('scaling', MinMaxScaler()),
                           ('classifier',LogisticRegression())])

pipeline.fit(X_train,y_train)


In [ ]:
# grid search
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

param_grid = {'classifier': [LogisticRegression(), KNeighborsClassifier(), GaussianNB(), 
                             DecisionTreeClassifier(), SVC(), LinearDiscriminantAnalysis()],
              'scaling': [MinMaxScaler(), StandardScaler()]              
             }

grid = GridSearchCV(pipeline, param_grid=param_grid, cv=5, n_jobs=-1,verbose=1, scoring='f1')
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
best_hyperparams = grid.best_params_
best_score = grid.best_score_
predictions= grid.predict(X_test)
print(f'Best test set f1_score:\n\t {best_score}\nAchieved with hyperparameters:\n\t {best_hyperparams}')
print( classification_report(y_test, predictions))

## 6. Deploy your model to cloud and test it with PostMan, BASH or Python

In [ ]:
#pickling 
import pickle 
